In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
DATA_FOLDER = "study_data"
DIRECTORY_TEMPLATE = {
    'retakes': 'retakes_number_after_{}_sem',
    'expulsion': 'expulsion_sem',
    'gpa': 'gpa_after_{}_sem'
}
INDEX_COLUMN = "id"
DB_PATH = "db.csv"

In [3]:
db = pd.read_csv(DB_PATH).drop_duplicates(subset=("name"), keep="first")
db.id = np.arange(len(db))
db = db.set_index(INDEX_COLUMN)
db.loc[db.admission_condition == "Общий конкурс", "admission_condition"] = "OK"
db.drop("name", axis=1)

,admission_year,admission_condition,exam_math,exam_inf,exam_rus,individual_achievements,exam_sum,student_mark_math,student_mark_inf,expulsion_sem,gpa_after_first_sem,gpa_after_second_sem,retakes_number_after_first_sem,retakes_number_after_second_sem
id,,,,,,,,,,,,,,
0,2017,БВИ,88.0,100.0,88.0,10.0,310.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,БВИ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,БВИ,98.0,94.0,86.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,БВИ,84.0,84.0,72.0,6.0,NaN,9.0,8.0,NaN,NaN,NaN,NaN,NaN
4,2017,БВИ,96.0,100.0,91.0,10.0,NaN,8.0,10.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2019,Контракт,86.0,84.0,89.0,8.0,283.0,4.0,8.0,NaN,NaN,NaN,NaN,NaN
309,2019,Контракт,92.0,91.0,94.0,4.0,281.0,8.0,3.0,NaN,NaN,NaN,NaN,NaN
310,2019,Контракт,88.0,91.0,89.0,10.0,278.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN


In [4]:
years = (2017, 2018, 2019)
semesters_count = {2017: 7, 2018: 5, 2019: 3}
semester_names = ("first", "second")

In [5]:
from math import isnan

# Fill expulsion sem

In [6]:
current_category = DIRECTORY_TEMPLATE["expulsion"]
directory = os.path.join(DATA_FOLDER, current_category)
for year in years:
    for semester in range(1, semesters_count[year] + 1):
        df = pd.read_csv(
            os.path.join(directory, "{}_{}_{}.csv".format(current_category, semester, year)), 
            sep=';',
            usecols=["Студент"]
        )
        not_found = 0
        repeat = 0
        success = 0
        for name in df.Студент:
            rows = db.loc[db.name == name, current_category]
            if rows.size != 1:
                not_found += 1
                continue
            if not isnan(rows.iloc[0]):
                repeat += 1
                continue
            success += 1
            db.loc[db.name == name, current_category] = semester
        print(f"Year {year}, sem {semester}: {not_found} not found, {repeat} repeated, {success} success")

Year 2017, sem 1: 2 not found, 0 repeated, 6 success
Year 2017, sem 2: 3 not found, 4 repeated, 1 success
Year 2017, sem 3: 3 not found, 0 repeated, 5 success
Year 2017, sem 4: 6 not found, 10 repeated, 6 success
Year 2017, sem 5: 7 not found, 16 repeated, 0 success
Year 2017, sem 6: 6 not found, 16 repeated, 2 success
Year 2017, sem 7: 6 not found, 18 repeated, 1 success
Year 2018, sem 1: 7 not found, 0 repeated, 5 success
Year 2018, sem 2: 16 not found, 4 repeated, 7 success
Year 2018, sem 3: 13 not found, 11 repeated, 5 success
Year 2018, sem 4: 15 not found, 15 repeated, 4 success
Year 2018, sem 5: 15 not found, 18 repeated, 5 success
Year 2019, sem 1: 21 not found, 12 repeated, 6 success
Year 2019, sem 2: 26 not found, 19 repeated, 3 success
Year 2019, sem 3: 16 not found, 10 repeated, 2 success


# Fill gpa

In [7]:
def fill_gpa(semester: int):
    global db, years, semester_names
    current_category = DIRECTORY_TEMPLATE["gpa"].format(semester_names[semester - 1])
    directory = os.path.join(DATA_FOLDER, current_category)
    for year in years:
        df = pd.read_csv(
            os.path.join(directory, "{}_{}.csv".format(current_category, year)), 
            sep=';',
            usecols=["Обучающийся", "Ср.балл по 10 балльной шкале"]
        )
        not_found = 0
        repeat = 0
        success = 0
        for name, gpa in df.to_numpy():
            if type(gpa) is float and isnan(gpa):
                continue
            rows = db.loc[db.name == name, current_category]
            if rows.size != 1:
                not_found += 1
                continue
            if not isnan(rows.iloc[0]):
                repeat += 1
                continue
            success += 1
            db.loc[db.name == name, current_category] = float(gpa.replace(',', '.'))
        print(f"Year {year}: {not_found} not found, {repeat} repeated, {success} success")

In [8]:
for semester in (1, 2):
    fill_gpa(semester)

Year 2017: 97 not found, 0 repeated, 95 success
Year 2018: 284 not found, 2 repeated, 98 success
Year 2019: 343 not found, 2 repeated, 111 success
Year 2017: 98 not found, 0 repeated, 83 success
Year 2018: 267 not found, 1 repeated, 88 success
Year 2019: 329 not found, 1 repeated, 110 success


# Fill retakes

In [9]:
def fill_retakes(semester: int):
    global db, years, semester_names
    current_category = DIRECTORY_TEMPLATE["retakes"].format(semester_names[semester - 1])
    directory = os.path.join(DATA_FOLDER, current_category)
    for year in years:
        df = pd.read_csv(
            os.path.join(directory, "{}_{}.csv".format(current_category, year)), 
            sep=';',
            usecols=["Студент", "Количество долгов"]
        )
        not_found = 0
        repeat = 0
        success = 0
        for name, retakes_number in df.to_numpy():
            rows = db.loc[db.name == name, current_category]
            if rows.size != 1:
                not_found += 1
                continue
            if not isnan(rows.iloc[0]):
                repeat += 1
                continue
            success += 1
            db.loc[db.name == name, current_category] = float(retakes_number)
        print(f"Year {year}: {not_found} not found, {repeat} repeated, {success} success")

In [10]:
for semester in (1, 2):
    fill_retakes(semester)

Year 2017: 84 not found, 0 repeated, 17 success
Year 2018: 422 not found, 0 repeated, 24 success
Year 2019: 376 not found, 2 repeated, 17 success
Year 2017: 87 not found, 0 repeated, 14 success
Year 2018: 442 not found, 0 repeated, 37 success
Year 2019: 481 not found, 2 repeated, 43 success


# Update db

In [11]:
db.drop("name", axis=1).to_csv("new_db.csv")

# Add olymp_info

In [12]:
olymp = pd.read_csv("olymp_info.csv")
olymp.drop("name", axis=1)

,admission_condition,math_olymp_100,inf_olymp_100,math_olymp_bvi,inf_olymp_bvi
0,БВИ,0,1,0,1
1,БВИ,0,1,0,1
2,БВИ,1,0,1,0
3,БВИ,1,0,1,0
4,БВИ,0,1,0,1
...,...,...,...,...,...
311,Контракт,0,1,0,0
312,Контракт,0,0,0,0
313,Контракт,0,0,0,0
314,Контракт,0,0,0,0


In [13]:
db = db.join(olymp[["math_olymp_100", "inf_olymp_100", "math_olymp_bvi", "inf_olymp_bvi"]])
db.drop("name", axis=1)

,admission_year,admission_condition,exam_math,exam_inf,exam_rus,individual_achievements,exam_sum,student_mark_math,student_mark_inf,expulsion_sem,gpa_after_first_sem,gpa_after_second_sem,retakes_number_after_first_sem,retakes_number_after_second_sem,math_olymp_100,inf_olymp_100,math_olymp_bvi,inf_olymp_bvi
id,,,,,,,,,,,,,,,,,,
0,2017,БВИ,88.0,100.0,88.0,10.0,310.0,9.0,NaN,NaN,7.17,6.53,NaN,NaN,0,1,0,1
1,2017,БВИ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,7.93,NaN,NaN,0,1,0,1
2,2017,БВИ,98.0,94.0,86.0,4.0,NaN,NaN,NaN,NaN,9.17,8.73,NaN,NaN,1,0,1,0
3,2017,БВИ,84.0,84.0,72.0,6.0,NaN,9.0,8.0,NaN,7.00,7.50,NaN,NaN,1,0,1,0
4,2017,БВИ,96.0,100.0,91.0,10.0,NaN,8.0,10.0,NaN,9.17,8.87,NaN,NaN,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2019,Контракт,86.0,84.0,89.0,8.0,283.0,4.0,8.0,NaN,5.14,6.35,1.0,NaN,0,0,0,0
309,2019,Контракт,92.0,91.0,94.0,4.0,281.0,8.0,3.0,NaN,6.29,6.29,NaN,NaN,0,0,0,0
310,2019,Контракт,88.0,91.0,89.0,10.0,278.0,4.0,4.0,1.0,NaN,NaN,NaN,NaN,1,0,0,0


In [14]:
db.drop("name", axis=1).to_csv("with_olymp_db.csv")